# UbiOps / Whylabs
This is a cookbook that show cases an example integration between UbiOps and Whylabs. In this cookbook we will train a model, build it and deploy it to the UbiOps environment while at each step of the way using Whylogs to log our data for monitoring and gaining interesting insights.

## Creating the model
This model is based on [this kaggle example](https://www.kaggle.com/cdabakoglu/heart-disease-classifications-machine-learning). It uses the [Heart Disease UCI dataset](https://www.kaggle.com/ronitf/heart-disease-uci).

This will be a very simplistic model to classify people into people who have a heart disease and people who do not which could be helpful to predict when you should go to the hospital to get checked or to know to estimate how many doctors will be neede in the staff of a hospital.

**First we will install our dependencies**

In [ ]:
 import sys 
!{sys.executable} -m pip install -U pip
!{sys.executable} -m pip install pandas --user
!{sys.executable} -m pip install sklearn --user
!{sys.executable} -m pip install whylogs==0.4.7.dev1 --user # Version needed for lib to work

In [ ]:
%load model/model.py

# #TODO 
### Add screenshots from the whylabs env for interesting findings when data shows up

## Creating UbiOps deployment
Now that we have built our model and saved it let's create a ubiops

**Let's first authenticate with our UbiOps environment**

In [ ]:
API_TOKEN = "<API_TOKEN>" # Make sure this is in the format "Token token-code"
PROJECT_NAME = "<PROJECT_NAME>"
DEPLOYMENT_NAME = 'heart-disease-classifier'
DEPLOYMENT_VERSION = 'v1'

WHYLABS_API_KEY= "<WHYLABS_API_KEY>"
WHYLABS_DEFAULT_ORG_ID = "<WHYLABS_DEFAULT_ORG_ID>"
WHYLABS_DEFAULT_DATASET_ID = "<WHYLABS_DEFAULT_DATASET_ID>"

# Import all necessary libraries
import shutil
import os
import ubiops

client = ubiops.ApiClient(ubiops.Configuration(api_key={'Authorization': API_TOKEN}, 
                                               host='https://api.ubiops.com/v2.1'))
api = ubiops.CoreApi(client)

**Create the deployment**

In [ ]:
%load deployment_folder/deployment.py

**Deploy to our UbiOps environment**

In [ ]:
# Create the deployment
deployment_template = ubiops.DeploymentCreate(
    name=DEPLOYMENT_NAME,
    description='Heart Disease classification',
    input_type='structured',
    output_type='structured',
    input_fields=[
        ubiops.DeploymentInputFieldCreate(
            name='data',
            data_type='blob',
        ),
    ],
    output_fields=[
        ubiops.DeploymentOutputFieldCreate(
            name='prediction',
            data_type='blob'
        ),
    ],
    labels={"demo": "whylabs"}
)

api.deployments_create(
    project_name=PROJECT_NAME,
    data=deployment_template
)

# Create the version
version_template = ubiops.DeploymentVersionCreate(
    version=DEPLOYMENT_VERSION,
    language='python3.6',
    memory_allocation=512,
    minimum_instances=0,
    maximum_instances=1,
    maximum_idle_time=1800 # = 30 minutes
)

api.deployment_versions_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=version_template
)

# Create environment variables for whylabs
api.deployment_environment_variables_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=ubiops.EnvironmentVariableCreate(
        name="WHYLABS_API_KEY",
        value=WHYLABS_API_KEY,
        secret=True
    )
)

api.deployment_environment_variables_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=ubiops.EnvironmentVariableCreate(
        name="WHYLABS_DEFAULT_ORG_ID",
        value=WHYLABS_DEFAULT_ORG_ID,
        secret=True
    )
)

api.deployment_environment_variables_create(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    data=ubiops.EnvironmentVariableCreate(
        name="WHYLABS_DEFAULT_DATASET_ID",
        value=WHYLABS_DEFAULT_DATASET_ID,
        secret=True
    )
)

# Zip the deployment package
shutil.make_archive('deployment_folder', 'zip', '.', 'deployment_folder')

# Upload the zipped deployment package
file_upload_result =api.revisions_file_upload(
    project_name=PROJECT_NAME,
    deployment_name=DEPLOYMENT_NAME,
    version=DEPLOYMENT_VERSION,
    file='deployment_folder.zip'
)

## Conclusion
We have now trained a model, used whylabs in the process to gain some insight into our training data, saved the AI model file and used it to create a deployment in our UbiOps environment which is now ready to receive requests and logs each request data to whylabs so you can spot the kind of data coming in and be able to improve on your model in the future.